# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f87dc79a8b0>
├── 'a' --> tensor([[-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819]])
└── 'x' --> <FastTreeValue 0x7f87dc79a820>
    └── 'c' --> tensor([[ 0.1617,  0.3107, -0.3233,  1.0049],
                        [-0.0978, -0.5819, -0.1908,  0.4174],
                        [-0.8782, -2.3492,  0.2004,  0.2164]])

In [4]:
t.a

tensor([[-2.9016,  0.6439,  1.7135],
        [-1.1963,  1.1351,  0.4819]])

In [5]:
%timeit t.a

59.8 ns ± 0.081 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f87dc79a8b0>
├── 'a' --> tensor([[-0.3378, -0.3343, -0.7482],
│                   [-0.9541, -0.2262, -1.0163]])
└── 'x' --> <FastTreeValue 0x7f87dc79a820>
    └── 'c' --> tensor([[ 0.1617,  0.3107, -0.3233,  1.0049],
                        [-0.0978, -0.5819, -0.1908,  0.4174],
                        [-0.8782, -2.3492,  0.2004,  0.2164]])

In [7]:
%timeit t.a = new_value

57.6 ns ± 0.0477 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.9016,  0.6439,  1.7135],
               [-1.1963,  1.1351,  0.4819]]),
    x: Batch(
           c: tensor([[ 0.1617,  0.3107, -0.3233,  1.0049],
                      [-0.0978, -0.5819, -0.1908,  0.4174],
                      [-0.8782, -2.3492,  0.2004,  0.2164]]),
       ),
)

In [10]:
b.a

tensor([[-2.9016,  0.6439,  1.7135],
        [-1.1963,  1.1351,  0.4819]])

In [11]:
%timeit b.a

55.3 ns ± 0.0638 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6719, -1.2582, -1.1667],
               [ 0.2363,  0.3939,  0.3682]]),
    x: Batch(
           c: tensor([[ 0.1617,  0.3107, -0.3233,  1.0049],
                      [-0.0978, -0.5819, -0.1908,  0.4174],
                      [-0.8782, -2.3492,  0.2004,  0.2164]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.0819 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 0.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 22.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8725ab1af0>
├── 'a' --> tensor([[[-2.9016,  0.6439,  1.7135],
│                    [-1.1963,  1.1351,  0.4819]],
│           
│                   [[-2.9016,  0.6439,  1.7135],
│                    [-1.1963,  1.1351,  0.4819]],
│           
│                   [[-2.9016,  0.6439,  1.7135],
│                    [-1.1963,  1.1351,  0.4819]],
│           
│                   [[-2.9016,  0.6439,  1.7135],
│                    [-1.1963,  1.1351,  0.4819]],
│           
│                   [[-2.9016,  0.6439,  1.7135],
│                    [-1.1963,  1.1351,  0.4819]],
│           
│                   [[-2.9016,  0.6439,  1.7135],
│                    [-1.1963,  1.1351,  0.4819]],
│           
│                   [[-2.9016,  0.6439,  1.7135],
│                    [-1.1963,  1.1351,  0.4819]],
│           
│                   [[-2.9016,  0.6439,  1.7135],
│                    [-1.1963,  1.1351,  0.4819]]])
└── 'x' --> <FastTreeValue 0x7f8725ab16d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.3 µs ± 152 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f87dc80d550>
├── 'a' --> tensor([[-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819],
│                   [-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819],
│                   [-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819],
│                   [-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819],
│                   [-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819],
│                   [-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819],
│                   [-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819],
│                   [-2.9016,  0.6439,  1.7135],
│                   [-1.1963,  1.1351,  0.4819]])
└── 'x' --> <FastTreeValue 0x7f87dc80df70>
    └── 'c' --> tensor([[ 0.1617,  0.3107, -0.3233,  1.0049],
                        [-0.0978, -0.5819, -0.1908,  0.4174],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 58 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.2 µs ± 59.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.1617,  0.3107, -0.3233,  1.0049],
                       [-0.0978, -0.5819, -0.1908,  0.4174],
                       [-0.8782, -2.3492,  0.2004,  0.2164]],
              
                      [[ 0.1617,  0.3107, -0.3233,  1.0049],
                       [-0.0978, -0.5819, -0.1908,  0.4174],
                       [-0.8782, -2.3492,  0.2004,  0.2164]],
              
                      [[ 0.1617,  0.3107, -0.3233,  1.0049],
                       [-0.0978, -0.5819, -0.1908,  0.4174],
                       [-0.8782, -2.3492,  0.2004,  0.2164]],
              
                      [[ 0.1617,  0.3107, -0.3233,  1.0049],
                       [-0.0978, -0.5819, -0.1908,  0.4174],
                       [-0.8782, -2.3492,  0.2004,  0.2164]],
              
                      [[ 0.1617,  0.3107, -0.3233,  1.0049],
                       [-0.0978, -0.5819, -0.1908,  0.4174],
                       [-0.8782, -2.3492,  0.2004,  0.2164]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 189 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.1617,  0.3107, -0.3233,  1.0049],
                      [-0.0978, -0.5819, -0.1908,  0.4174],
                      [-0.8782, -2.3492,  0.2004,  0.2164],
                      [ 0.1617,  0.3107, -0.3233,  1.0049],
                      [-0.0978, -0.5819, -0.1908,  0.4174],
                      [-0.8782, -2.3492,  0.2004,  0.2164],
                      [ 0.1617,  0.3107, -0.3233,  1.0049],
                      [-0.0978, -0.5819, -0.1908,  0.4174],
                      [-0.8782, -2.3492,  0.2004,  0.2164],
                      [ 0.1617,  0.3107, -0.3233,  1.0049],
                      [-0.0978, -0.5819, -0.1908,  0.4174],
                      [-0.8782, -2.3492,  0.2004,  0.2164],
                      [ 0.1617,  0.3107, -0.3233,  1.0049],
                      [-0.0978, -0.5819, -0.1908,  0.4174],
                      [-0.8782, -2.3492,  0.2004,  0.2164],
                      [ 0.1617,  0.3107, -0.3233,  1.0049],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
